<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/06a_PytorchTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tarea: responder donde dice **PREGUNTA**

## ¿Qué es pytorch?

[PyTorch](https://pytorch.org/) es una librería de deep learning de código abierto basada en Python.

Tiene tres componentes principales:

* Es una librería de **tensores**, similar a numpy, pero con soporte para GPU.

* Es un **motor de diferenciación automática** (autograd): calcula gradientes de funciones automáticamente, con el objetivo de simplificar el _backpropagation_ y la optimización de modelos.

* Es una librería de **deep learning**: tiene módulos para diseñar y entrenar redes neuronales profundas--modelos preentrenados, funciones de pérdida, optimizadores, etc.

## Instalación

PyTorch se puede instalar como cualquier otra librería de Python.




In [ ]:
!pip install -q torch watermark
# usamos watermark para hacer un print de las versiones que usamos

In [ ]:
%load_ext watermark

In [ ]:
%watermark -vp torch

In [ ]:
# Verificar GPU:
import torch

print(torch.cuda.is_available())

Si devuelve False, significa que no hay GPU compatible o no es reconocida.

Aunque no es obligatorio tener GPU para usar PyTorch, **acelera enormemente** las operaciones matriciales que usamos en la inferencia y entrenamiento.

En google colab, usar `Runtime > Change runtime type` para seleccionar una GPU.

-----------

Al ser una librería grande con soporte para GPU, la instalación puede requerir algunas consideraciones adicionales si la hacemos en otro entorno.

En particular, pytorch con GPU requiere **CUDA**, un API desarrollada por NVIDIA que permite aprovechar la GPU no solo para gráficos, sino también para cómputo general.

La instalación de CUDA puede ser problemática. Fuera de colab, puede ser útil crear un virtual environment para manejar esto. Por ejemplo de conda:

```bash
conda create --name my-env python=3.11 -y &&
conda activate my-env &&
conda install pytorch pytorch-cuda=12.4 -c pytorch -c nvidia -y
# pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126
```

También pueden ver en [la página oficial](https://pytorch.org) el comando adecuado según el sistema operativo y versión de CUDA.

Respecto de la versión de Python: muchas librerías no soportan de inmediato la versión más reciente de Python, así que se recomienda usar una versión una o dos ediciones anteriores (por ejemplo, si la última es 3.13, usar 3.11 o 3.12). En este caso usamos la que viene por defecto en colab.

## Tensores

Los **tensores** son un objeto matemático que **generaliza vectores y matrices** a dimensiones más altas. El orden o **rango** de los tensores indica el número de dimensiones.

* Un escalar es un tensor de rango 0
* Un vector es un tensor de rango 1
* Una matriz es un tensor de rango 2
* Tensores de orden superior se nombran como tensores 3D, 4D, etc.

En computación, los tensores funcionan como **contenedores de datos multidimensionales**, donde cada dimensión representa una característica distinta.

Con PyTorch podemos crearlos, manipularlos y hacer cálculos de forma eficiente. Funcionan como los _arrays_ de numpy, pero con funcionalidades adicionales útiles para hacer deep learning: **diferenciación automática**, y **soporte para GPU**.

In [ ]:
# Creación de tensores

tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2], [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [ ]:
# Tipos de datos

tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

In [ ]:
import numpy as np

np_array = np.array([1.0, 2.0, 3.0])
print(np_array.dtype)

Pytorch usa por defecto float32 en lugar de float64 como numpy porque está optimizado para GPUs, donde **float32** es más rápido y consume menos memoria, y esta precisión es suficiente para hacer deep learning.

In [ ]:
# Cambiar la precisión:

floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

In [ ]:
# Creación de un tensor
tensor2d = torch.tensor([[1, 2, 3],
                         [4, 5, 6]])
tensor2d

In [ ]:
# Consultar la forma (shape)

print(tensor2d.shape)


**PREGUNTA 1** ¿Cuál es la diferencia entre los atributos .shape y .ndim?


In [ ]:
# Cambiar la forma

tensor2d.view(3, 2)

In [ ]:
# Trasponer:

tensor2d.T

In [ ]:
# Multiplicar matrices:

print(tensor2d.matmul(tensor2d.T))
print(tensor2d @ tensor2d.T)

Muchas veces pyotorch ofrece varias formas de hacer lo mismo, porque combina convenciones del antiguo Torch y de numpy. e.g. reshape y view, size y shape.

## Modelos y diferenciación automática

El motor de diferenciación automática de pytorch, **autograd**, se usa para calcular gradientes automáticamente en grafos computacionales.

Un [**grafo computacional**](https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/) es un grafo dirigido que representa operaciones matemáticas. En deep learning, un grafo computacional representa la secuencia de **cálculos que se hacen para obtener la salida de una red neuronal**. Este grafo sirve para calcular los gradientes usados en **backpropagation**, el algoritmo de entrenamiento de redes neuronales.


Por ejemplo, en una regresión logística, que puede verse como una red neuronal de una sola capa, los cálculos pueden representarse como un grafo: la entrada o feature se multiplica por un peso, se suma un sesgo, se aplica una función de activación (sigmoide) y luego se compara con la etiqueta verdadera para calcular la pérdida.

In [ ]:
# Ejemplo: regresión logística con un feature de entrada y una salida

import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1]) # entrada (feature)
w1 = torch.tensor([2.2]) # weight
b = torch.tensor([0.0])  # bias

z = x1 * w1 + b
a = torch.sigmoid(z)     # activación y output

loss = F.binary_cross_entropy(a, y)
print(loss)

**Backpropagation** aplica la regla de la cadena de derivadas "de derecha a izquierda" en el grafo, comenzando en la capa de salida y la pérdida, y yendo hacia atrás hasta la entrada. Esto permite calcular **cómo cambia la pérdida con respecto a cada parámetro** (pesos y sesgos), que a su vez sirve para **actualizar los parámetros** y mejorar el modelo durante el entrenamiento.

* Las **derivadas parciales** miden cómo cambia una función respecto a una de sus variables.
* Un **gradiente** es un vector que contiene todas las derivadas parciales de una función multivariable.
* La **regla de la cadena** permite combinar estas derivadas en el grafo para obtener los gradientes.

Si alguno de los nodos terminales que usamos tiene el atributo `requires_grad=True`, entonces se construye internamente un grafo computacional en segundo plano. Luego, al llamar a la función `grad`, podemos calcular el gradiente de la función de pérdida con respecto a cualquier parámetro del modelo.

In [ ]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

**PREGUNTA 2**: ¿cuáles son los parámetros o pesos en el ejemplo inmediatamente anterior?

In [ ]:
print(grad_L_w1)
print(grad_L_b)

**PREGUNTA 3**: ¿qué representan los dos valores inmediatamente anteriores?

Por defecto, PyTorch destruye el grafo computacional después de calcular los gradientes para liberar memoria. Si vamos a reutilizar ese grafo, podemos usar `retain_graph=True` para que se mantenga en memoria.

En la práctica, no hace falta usar `grad` de manera "manual"--pytorch tiene herramientas de más alto nivel para automatizar este proceso. Podemos usar el **método `.backward()`** sobre la pérdida para calcular automáticamente los gradientes de todos los nodos hoja del grafo, almacenándolos en los atributos `.grad` de los tensores correspondientes.

In [ ]:
loss.backward()

print(w1.grad)
print(b.grad)

En resumen: **autograd** registra todas las operaciones con tensores, construye automáticamente el grafo computacional en segundo plano y, cuando llamamos al método `.backward()` sobre la pérdida, calcula automáticamente los gradientes de todos los parámetros involucrados. No necesitamos calcular derivadas ni gradientes a mano--pytorch se encarga de todo de forma automática.

## Deep learning

Al implementar una red neuronal en pytorch, normalmente instanciamos una subclase de la clase  `torch.nn.Module` para definir nuestra propia arquitectura personalizada. `Module` facilita la construcción y entrenamiento de modelos, por ejemplo, encapsulando capas y operaciones, y llevando un seguimiento de los pesos del modelo.

Dentro de la subclase, definimos las capas de la red en el método `__init__` y especificamos cómo interactúan en el método **`forward`**. Este método define cómo los datos de entrada pasan a través de la red y se combinan en un grafo computacional.

In [ ]:
# Ejemplo de un perceptrón multicapa (MLP):

class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

# Sequential() no es obligatorio pero ayuda a evitar código repetido cuando hay muchas operaciones
# porque podemos llamar directamente a layers()

In [ ]:
# Creamos una instancia de la clase
model = NeuralNetwork(50, 3)

**PREGUNTA 4**: ¿Cuántas capas ocultas y de salida tiene este modelo?

In [ ]:
# Resumen de la estructura del modelo:
print(model)

In [ ]:
# Cantidad de parámetros del modelo
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Cantidad de parámetros entrenables:", num_params)



**PREGUNTA 5** ¿qué significa que un parámetro es entrenable?

In [ ]:
# Acceder a una matriz de pesos específica:
print(model.layers[0].weight)
print(model.layers[0].weight.shape)

In [ ]:
# Inicialización de pesos reproducible:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

**PREGUNTA 6**: ¿por qué se suelen inicializar de manera aleatoria los pesos de una red?

In [ ]:
# Método forward:
torch.manual_seed(123)

X = torch.rand((1, 50)) # entrada ficticia
out = model(X) # forward
print(out)

`grad_fn=...` indica la última función usada para calcular una variable en el grafo computacional.

`<AddmmBackward0>` significa que el tensor se creó mediante una multiplicación de matrices (mm) y suma (add). Pytorch usa esta información cuando calcula gradientes durante la retropropagación.

Si solo queremos usar una red para hacer predicciones, no hace falta construir el grafo computacional para seguir los gradientes--esto implica cálculos innecesarios y consumo de memoria.

Para desactivar estas operaciones en la **inferencia**, usamos el context manager `torch.no_grad()` o `torch.inference_mode()` para ahorrar memoria y cómputo.

In [ ]:
with torch.no_grad():
    out = model(X)
print(out)

En problemas de clasificación, solemos programar los modelos para que devuelvan directamente los **logits** (las salidas de la última capa) sin pasarlas por una función de activación como softmax o sigmoidea.

Esto es porque las funciones de pérdida de pytorch ya combinan la operación softmax con la cross entropy por eficiencia y estabilidad numérica.

Entonces, para calcular probabilidades de pertenencia a clases para las predicciones, tenemos que usar explícitamente la función softmax.

In [ ]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

**PREGUNTA 7**: ¿Cuál es la diferencia entre softmax y sigmoidea?

## Procesamiento de datos

Para entrenar modelos necesitamos maneras de cargar y procesar los datos. Hay tres componentes fundamentales para esto:

* La clase `Dataset` para instanciar objetos que definen cómo se carga cada registro
* La clase `DataLoader` para manejar cómo se mezclan los datos y se agrupan en tandas o **batches**
* Una función `collate` que aplica procesamiento a los batches e.g. tokenización, padding, etc.

In [ ]:
# Supongamos que ya tenemos features y labels listos para entrenar y evaluar
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

# las etiquetas deben comenzar con 0, y el valor máximo no debe exceder el número de salidas menos 1

Los tres componentes principales de un **`Dataset`** son:

* En `__init__` configuramos atributos a los que podemos acceder más adelante en `__getitem__` y `__len__`. Estos atributos podrían ser rutas de archivos, objetos de archivos, conectores de bases de datos, etc. En este caso, como tenemos tensores en memoria, simplemente asignamos `X` y `y` a estos atributos.
* En `__getitem__` definimos cómo devolver exactamente un elemento del dataset mediante un índice. Esto significa devolver los features y la etiqueta de clase de un solo ejemplo.
* En `__len__` definimos cómo obtener la longitud del dataset.

In [ ]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [ ]:
len(train_ds)

Usamos la clase **`DataLoader`** para tomar muestras e iterar sobre el dataset.

El loader recorre todo el dataset de entrenamiento visitando cada ejemplo exactamente una vez. Esto se conoce como un **epoch** de entrenamiento.

Si fijamos la semilla del generador aleatorio, obtenemos siempre el mismo orden de las muestras en la primera ejecución. Si iteramos una segunda vez sobre el dataset, el orden cambia--esto evita que las redes neuronales caigan en ciclos repetitivos durante el entrenamiento.

In [ ]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

test_ds = ToyDataset(X_test, y_test)
test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [ ]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Como usamos batch_size=2, el tercer batch solo contiene un ejemplo porque tenemos 5 ejemplos en total. En la práctica, tener un último batch más chico puede afectar la convergencia del entrenamiento--podemos eliminar el último batch de cada epoch con `drop_last=True`.

In [ ]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [ ]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Con `num_workers` podemos paralelizar la carga y el preprocesamiento de datos:

* Cuando `num_workers=0`, la carga de datos ocurre en el proceso principal. Esto puede generar cuellos de botella durante el entrenamiento de modelos grandes en GPU porque la CPU debe cargar/preprocesar los datos mientras la GPU queda inactiva esperando.
* Con `num_workers>0`, se lanzan varios procesos en paralelo para cargar los datos, permitiendo que el proceso principal se concentre en entrenar el modelo y aprovechar mejor los recursos. El loader puede ir preparando los siguientes batches en segundo plano.

La configuración ideal depende del hardware y código específicos--`num_workers=4` es un buen punto de partida para datos y/o modelos grandes.

Finalmente, la **`collate` function** (función de ensamblado) define cómo se combinan las muestras individuales de un dataset en un batch dentro del DataLoader. Es decir, el collator se encarga de cómo se ve cada batch cuando llega al entrenamiento.

Por defecto, pytorch usa `default_collate`, que simplemente apila los tensores (stack) para formar un batch.

En algunos casos los datos necesitan cierto preprocesamiento. Por ejemplo, si las muestras tienen longitudes variables podemos aplicar padding u otra transformación antes de pasar los datos al modelo.

Veamos un ejemplo de juguete en el que rellenamos muestras de distinta longitud con 0s hasta que todas tengan la misma longitud dentro del lote.

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# Dataset de ejemplo con secuencias de distinta longitud
class VarLenDataset(Dataset):
    def __init__(self):
        self.data = [
            torch.tensor([1, 2, 3]),
            torch.tensor([4, 5]),
            torch.tensor([6]),
            torch.tensor([7, 8, 9, 10])
        ]
        self.labels = torch.tensor([0, 1, 0, 1])  # etiquetas binarias

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def __len__(self):
        return len(self.data)

# Collator personalizado con padding
def pad_collator(batch):
    # batch = [(seq1, label1), (seq2, label2), ...]
    sequences = [item[0] for item in batch]
    labels = torch.tensor([item[1] for item in batch])
    # Rellenar las secuencias al largo máximo del lote (padding)
    padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    # batch_first pone como primera dimensión el batch_size
    return padded, labels

dataset = VarLenDataset()
loader = DataLoader(dataset, batch_size=2, collate_fn=pad_collator, shuffle=True)

for idx, (x, y) in enumerate(loader):
    print(f"Batch {idx+1}:")
    print("Secuencias con padding:\n", x)
    print("Labels:", y, "\n")

## Entrenamiento

Veamos un loop de entrenamiento clásico.

* Usamos un **optimizador** de descenso de gradiente estocástico (SGD) con una **tasa de aprendizaje** (lr) de 0.5. La tasa de aprendizaje es un hiperparámetro i.e. en la práctica debemos ajustarla observando la pérdida.
* El **número de épocas** es otro hiperparámetro a elegir.
* A veces usamos un tercer conjunto de datos, de **validación**, para encontrar la configuración óptima de hiperparámetros. Normalmente usamos el set de validación muchas veces, cosa que no debemos hacer con el de test.
* `model.train()` y `model.eval()` se usan para poner el modelo en **modo entrenamiento** y **modo evaluación**. Esto es necesario para componentes que se comportan de manera diferente durante el entrenamiento y la inferencia, como dropout o batch normalization. En este casos usarlos es redundante, pero es una buena práctica incluirlos siempre.
* Pasamos los logits directamente a la **función de pérdida** cross_entropy, que ya aplica la función softmax internamente.
* Al llamar **`loss.backward()`** se calculan los gradientes en el grafo computacional-
* El método `optimizer.step()` usa los gradientes para **actualizar los parámetros del modelo** en pos de minimizar la pérdida. En el caso de SGD, esto significa multiplicar los gradientes por la tasa de aprendizaje y restarlo a los parámetros.
* Es importante incluir una llamada a `optimizer.zero_grad()` en cada actualización para **reiniciar los gradientes** a cero--de lo contrario, los gradientes se acumulan, lo cual puede ser no deseado.

In [ ]:
import torch.nn.functional as F


torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")

    model.eval()
    # Evaluación opcional acá

In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

In [ ]:
# Obtener las probabilidades
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

In [ ]:
# Obtener predicciones de etiquetas
predictions = torch.argmax(probas, dim=1)
print(predictions)

In [ ]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

**PREGUNTA 8**: ¿Por qué da el mismo resultado hacer el argmax() sobre las probabilidades o los logits?

In [ ]:
# Accuracy en los datos de entrenamiento
print(predictions == y_train)
print(torch.sum(predictions == y_train))
print(torch.mean((predictions == y_train).float()))

In [ ]:
# Implementado como función para escalar a datasets de cualquier tamaño:

def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [ ]:
compute_accuracy(model, train_loader)

In [ ]:
compute_accuracy(model, test_loader)

El `state_dict` de un modelo es un diccionario que mapea cada capa del modelo a sus parámetros entrenables. Para **guardar un modelo**, guardamos esto.


In [ ]:
torch.save(model.state_dict(), "model.pt")
# "model.pt" es un nombre de archivo arbitrario
# Podemos usar cualquier nombre y extensión -- .pth y .pt son las convenciones más comunes.

Con `torch.load(...)` leemos el archivo y reconstruimos el diccionario que contiene los parámetros del modelo, mientras que `model.load_state_dict()` aplica estos parámetros al modelo, restaurando su estado en el momento en que lo guardamos.

Necesitamos una **instancia del modelo en memoria** para aplicar los parámetros guardados; i.e., la arquitectura `NeuralNetwork(2, 2)` debe coincidir exactamente con el modelo original guardado.

In [ ]:
model = NeuralNetwork(2, 2) # needs to match the original model exactly
model.load_state_dict(torch.load("model.pt", weights_only=True))

## Uso de GPU

En pytorch, un **device** es el dispositivo donde ocurren los cálculos y están los datos. CPU y GPU son los device que solemos usar.

Por defecto, pytorch usa CPU. Podemos usar el método `.to()` para mandar tensores a una GPU y hacer las operaciones ahí.

In [ ]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])

print(tensor_1 + tensor_2)

In [ ]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")

print(tensor_1 + tensor_2)

`device='cuda:0'` significa que los tensores están en la primera GPU. Si la máquina tiene varias GPUs, podemos especificar cuál GPU usar.

Si los tensores están en devices distintos, rompe:

In [ ]:
tensor_1 = tensor_1.to("cpu")
print(tensor_1 + tensor_2)

Entrenando con GPU:

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# Definir device:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Mandar modelo a device:
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        # Mandar datos a device:
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels) # Loss function

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")

    model.eval()

En este caso seguramente no haya mucha diferencia -- la **aceleración es abismal cuando usamos modelos grandes**.

Cuando los modelos son demasiado grandes, se suelen usar múltiples GPUs en lugar de una sola. Eso es un poco difícil de aplicar en el contexto de una notebook.

## Recursos:

* https://sebastianraschka.com/teaching/pytorch-1h
* https://docs.pytorch.org/tutorials